### Mod 1 Project --- Data Movie

In [1]:
#top 50 lucrative movies from  tn_movie_budgets and bom_movie_gross   (find their ratings)
# most (top50) popular movies from tmdb_movies
# highest(top 50) ratings from imdb_title_ratings

# select common movies from above list
#find their genre, directors

### import database

In [160]:
import pandas as pd
import numpy as np
import datetime as dt
import mysql.connector
import matplotlib.pyplot as plt
%matplotlib inline

In [161]:
con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='movies')
cur = con.cursor()

# Data Cleaning

### bom database

In [162]:
query = ('''
SELECT * FROM bom_movie_gross
''')
bom = pd.read_sql_query(query, con)

In [163]:
#drop Null value in domestic_gross
bom = bom.dropna(subset =['domestic_gross'])

#convert Null value in foreign_gross to 0
bom.foreign_gross = bom.foreign_gross.fillna(0)

#convert domestic_gross and foreign_gross from float to int
bom.domestic_gross = bom.domestic_gross.map(lambda x: int(x))   
bom.foreign_gross = bom.foreign_gross.map(lambda x: int(x))  

In [164]:
#remove spaces on both end of strings
bom.title = bom.title.map(lambda x: x.strip())
bom.studio = bom.studio.map(lambda x: x.strip())

In [165]:
bom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3359 entries, 0 to 3386
Data columns (total 5 columns):
title             3359 non-null object
studio            3359 non-null object
domestic_gross    3359 non-null int64
foreign_gross     3359 non-null int64
year              3359 non-null int64
dtypes: int64(3), object(2)
memory usage: 157.5+ KB


### tn database

In [166]:
query_tn = ('''
SELECT * FROM tn_movie_budgets
''')
tn = pd.read_sql_query(query_tn, con)

In [167]:
# convert release_date to date format
tn.release_date = tn.release_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y'))

# convert production_budget, domestic_gross, worldwide_gross to date format
tn.production_budget = tn.production_budget.map(lambda x: int(x.replace(',','').replace('$','')))
tn.domestic_gross = tn.domestic_gross.map(lambda x: int(x.replace(',','').replace('$','')))
tn.worldwide_gross = tn.worldwide_gross.map(lambda x: int(x.replace(',','').replace('$','')))

#add profit column
tn['profit'] = tn.worldwide_gross - tn.production_budget

In [168]:
#remove spaces on both end of strings
tn.movie = tn.movie.map(lambda x: x.strip())

In [169]:
tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 7 columns):
id                   5782 non-null int64
release_date         5782 non-null datetime64[ns]
movie                5782 non-null object
production_budget    5782 non-null int64
domestic_gross       5782 non-null int64
worldwide_gross      5782 non-null int64
profit               5782 non-null int64
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 316.3+ KB


### tmdb database

In [173]:
query_tmdb = ('''
SELECT * FROM tmdb_movies
''')
tmdb = pd.read_sql_query(query_tmdb, con)

In [174]:
query_tmdb_link = ('''
SELECT * FROM tmdb_imdb_link
''')
tmdb_link = pd.read_sql_query(query_tmdb_link, con)

In [175]:
#convert genre_ids to int
tmdb.genre_ids.map(lambda x: int(x))

#remove spaces on both end of strings
tmdb.original_language = tmdb.original_language.map(lambda x: x.strip())
tmdb.original_title = tmdb.original_title.map(lambda x: x.strip())
tmdb.title = tmdb.title.map(lambda x: x.strip())

In [176]:
#Join tmdb and tmdb_link tables
tmdb = tmdb.join(tmdb_link.set_index('tmdb_id'), on = 'id', how = 'left')

In [177]:
#clean the joined table
tmdb.imdb_id = tmdb.imdb_id.fillna(0).map(lambda x: int(x))

In [178]:
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26526 entries, 0 to 26516
Data columns (total 10 columns):
genre_ids            26526 non-null object
id                   26526 non-null int64
original_language    26526 non-null object
original_title       26526 non-null object
popularity           26526 non-null float64
release_date         26526 non-null object
title                26526 non-null object
vote_average         26526 non-null float64
vote_count           26526 non-null int64
imdb_id              26526 non-null int64
dtypes: float64(2), int64(3), object(5)
memory usage: 2.2+ MB


### rt database

In [179]:
query_rt_info = ('''
SELECT * FROM rt_movie_info
''')
rt_info = pd.read_sql_query(query_rt_info, con)

In [180]:
query_rt_reviews = ('''
SELECT * FROM rt_reviews
''')
rt_reviews = pd.read_sql_query(query_rt_reviews, con)

In [181]:
query_rt_link = ('''
SELECT * FROM rt_imdb_link
''')
rt_link = pd.read_sql_query(query_rt_link, con)

In [182]:
rt_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
id              1560 non-null int64
synopsis        1560 non-null object
rating          1560 non-null object
genre           1560 non-null object
director        1560 non-null object
writer          1560 non-null object
theater_date    1560 non-null object
dvd_date        1560 non-null object
currency        1560 non-null object
box_office      1560 non-null object
runtime         1560 non-null object
studio          494 non-null object
dtypes: int64(1), object(11)
memory usage: 146.3+ KB


#### rt_info cleanning

In [183]:
#replace empty value in theater_date and dvd_date with Jan 1,1000
rt_info['theater_date'].loc[rt_info.theater_date == ''] = 'Jan 1, 1000'
rt_info['dvd_date'].loc[rt_info.dvd_date == ''] = 'Jan 1, 1000'

In [184]:
#convert theater_date and dvd_date to datetime format
rt_info.theater_date = rt_info.theater_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y').date())
rt_info.dvd_date = rt_info.dvd_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y').date())

In [185]:
#clean box_office date
rt_info['box_office'].loc[rt_info.box_office == ''] = '0'
rt_info['box_office'] = rt_info.box_office.map(lambda x: int(x.replace(',', '')))

In [186]:
#clean runtime (replace empty value with 0)
rt_info['runtime'].loc[rt_info.runtime == ''] = '0'
rt_info['runtime'] = rt_info.runtime.map(lambda x: int(x.replace('minutes', '')))

In [187]:
#remove spaces on both end of strings
rt_info.synopsis = rt_info.synopsis.map(lambda x: x.strip())
rt_info.rating = rt_info.rating.map(lambda x: x.strip())
rt_info.genre = rt_info.genre.map(lambda x: x.strip())
rt_info.director = rt_info.director.map(lambda x: x.strip())
rt_info.writer = rt_info.writer.map(lambda x: x.strip())

In [188]:
rt_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
id              1560 non-null int64
synopsis        1560 non-null object
rating          1560 non-null object
genre           1560 non-null object
director        1560 non-null object
writer          1560 non-null object
theater_date    1560 non-null object
dvd_date        1560 non-null object
currency        1560 non-null object
box_office      1560 non-null int64
runtime         1560 non-null int64
studio          494 non-null object
dtypes: int64(3), object(9)
memory usage: 146.3+ KB


#### rt_reviews cleanning

In [189]:
#convert date to datetime format
rt_reviews.date = rt_reviews.date.map(lambda x: dt.datetime.strptime(x, '%B %d, %Y').date())

In [190]:
#convert top_critic to int
rt_reviews.top_critic = rt_reviews.top_critic.map(lambda x: int(x))

In [191]:
#remove spaces on both end of strings
rt_reviews.review = rt_reviews.review.map(lambda x: x.strip())
rt_reviews.rating = rt_reviews.rating.map(lambda x: x.strip())
rt_reviews.fresh = rt_reviews.fresh.map(lambda x: x.strip())
rt_reviews.critic = rt_reviews.critic.map(lambda x: x.strip())
rt_reviews.publisher = rt_reviews.publisher.map(lambda x: x.strip())

In [192]:
rt_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12300 entries, 0 to 12299
Data columns (total 8 columns):
id            12300 non-null int64
review        12300 non-null object
rating        12300 non-null object
fresh         12300 non-null object
critic        12300 non-null object
top_critic    12300 non-null int64
publisher     12300 non-null object
date          12300 non-null object
dtypes: int64(2), object(6)
memory usage: 768.8+ KB


#### rt_link cleaning

In [193]:
rt_link.year = rt_link.year.fillna(1000)

In [194]:
rt_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17468 entries, 0 to 17467
Data columns (total 6 columns):
imdbid         17468 non-null int64
rtid           17468 non-null int64
title          17468 non-null object
year           17468 non-null float64
mpaa_rating    17468 non-null object
runtime        17468 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 818.9+ KB


In [195]:
#replace empty space to year 1000 in year
rt_link.year = rt_link.year.fillna(1000)

#convert date to datetime format
rt_link.year = rt_link.year.map(lambda x: (int(x)))

In [196]:
#replace empty space in runtime to 0
rt_link['runtime'].loc[rt_link.runtime == ''] = '0'

#convert runtime to int format
rt_link.runtime = rt_link.runtime.map(lambda x: (int(x)))

In [197]:
rt_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17468 entries, 0 to 17467
Data columns (total 6 columns):
imdbid         17468 non-null int64
rtid           17468 non-null int64
title          17468 non-null object
year           17468 non-null int64
mpaa_rating    17468 non-null object
runtime        17468 non-null int64
dtypes: int64(4), object(2)
memory usage: 818.9+ KB


In [198]:
rt_link.shape

(17468, 6)

# Top 200 lucrative movies

### bom database

In [199]:
# get top 200 lucrative domestic movies
bom_top200_domestic = bom.sort_values('domestic_gross', ascending = False).head(200)   

### tn database

In [200]:
# get top 200 revenue domestic movies
tn_top200_domestic = tn.sort_values('domestic_gross',ascending=False).head(200)

# get top 200 revenue worldwide movies
tn_top200_world = tn.sort_values('worldwide_gross',ascending=False).head(200)

# get top 200 profit movies
tn_top200_profit = tn.sort_values('profit',ascending=False).head(200)